In [ ]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

In [ ]:
url = "https://lhr.nu.edu.pk/faculty/"
site = requests.get(url)
soup = BeautifulSoup(site.content, "html.parser")

In [ ]:
id = ["fsc","ee","cv","fsm","ss"] #Id of all departments from html of site
for i in id: #loop for each department
  data = soup.find("div", id = i)

  fac_name = data.find_all("h5", "text-center")# Names of Faculty Members
  names = [name.text for name in fac_name]

  fac_email = data.find_all("p", "mb-0 text-center")# Emails of Faculty Members
  emails = [email.text for email in fac_email]

  fac_des = data.find_all("p", "small text-center font-italic") # To get Designation and if faculty member is HEC supervised or not
  des = [des.text.strip().split('\n')[0] for des in fac_des]
  HEC = [True if (des.find('br')) else False for des in fac_des]

  fac_img = data.find_all("img")# Location of Faculty Member Image
  img = [img['src'] for img in fac_img]

  fac_department = data.find('h1', "mb-2 mt-3").text #Department

  fac_id = data.find_all("a","faculty-link")# Geting the ID of Facuty Member
  id = [link.get("href").split('/')[3] for link in fac_id]

  #Creating the Data frame which includes all featues of faculty members of specific department
  Dep_fac_data = pd.DataFrame({'ID': id,
                      'Name': names,
                      'Designation': des,
                      'HEC': HEC,
                      'Department': fac_department,
                      'Email': emails,
                      'Image': img})
  #String data in csv format with id of Department
  Dep_fac_data.to_csv(i+'.csv', index=False)

In [ ]:
id = ["fsc","ee","cv","fsm","ss"]
for i in id:#loop for each department
  data = pd.read_csv(i+'.csv')
  IDs = data['ID']#Geting the ID to visit page of each faculty member indvidually
  links = ["https://lhr.nu.edu.pk/fsc/facultyProfile/"+str(Id) for Id in IDs]#Creating list of links of all faculty members of specific department
  fac_Edu = []
  fac_Ext = []
  for link in links:
    fac_d = requests.get(link)
    fac_cont = BeautifulSoup(fac_d.content, "html.parser")#Getting the content of faculty members page

    edu_div = fac_cont.find("div", class_="col-lg-8 col-md-6 col-sm-12 text-justify")#Highest Education
    if edu_div:
        edu_item = edu_div.find("li")
        fac_Edu.append(edu_item.text.strip() if edu_item else "NA")
    else:
        fac_Edu.append("NA")

    fac_Ext.append(fac_cont.find("span","small").text.split(":")[1] if fac_cont.find("span","small") else "NA")#Extenstion of Faculty Member
  Fac_data = pd.DataFrame({'ID':IDs, 'Extention':fac_Ext, 'Highest Education':fac_Edu})#Creating Data Frame of Data of all faculty members of specific department
  Fac_data.to_csv(i+'2.csv', index=False)#Storing data with department ID

In [ ]:

Data = {}#Data of csv files containing Name,Email etc.
for i in id:
  data = pd.read_csv(i+'.csv')
  Data[i] = data

Data2 = {}#Data of csv files containing Extention and Education etc.
for i in id:
  data = pd.read_csv(i+'2.csv')
  Data2[i] = data

for i in Data:# Merging Data on basis of ID
  Data[i] = pd.merge(Data[i], Data2[i], on='ID', how='left')

Faculty_Data = pd.concat(Data.values(), ignore_index=True)#Creating a Dataframe which contain information of all departments of FAST NUCES Lahore
Faculty_Data.to_csv('fast_lahore_faculty.csv', index=False)